In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
%matplotlib inline 

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /Users/dungnd/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-6-25 Python-3.10.9 torch-2.0.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp2/weights/best.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /Users/dungnd/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-6-25 Python-3.10.9 torch-2.0.0 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [9]:
def pre_processing_img(crop_img):
    # grayscale region within bounding box
    gray = cv2.cvtColor(crop_img, cv2.COLOR_RGB2GRAY)

    # resize image to three times as large as original for better readability
    gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)

    # perform gaussian blur to smoothen image
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    # threshold the image using Otsus method to preprocess for tesseract
    ret, binary = cv2.threshold(blur, 127, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # perform another blur on character region
    result = cv2.medianBlur(binary, 5)

    return result

In [21]:
def filter_text(region, ocr_result, region_threshold=0.2):
    rectangle_size = region.shape[0]*region.shape[1]
    plate = []
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))
        if length*height / rectangle_size > region_threshold:
            plate.append(result)
    return plate

In [125]:
# img_path = os.path.join('data', 'images', 'CarLongPlate137.jpg')
img_path = os.path.join('/Users/dungnd/Desktop/Workspace/AI/license-plate-detection/server/test_img', 'test_17.jpeg')
# img_path = 'https://www.roadpol.eu/images/2022/SEP/Russian.jpg'
# img_path = 'https://image.vietnamnews.vn/uploadvnnews/Article/2017/6/15/5b3929415d1_wize16423115PM.jpg'
# img_path = 'https://media.urbanistnetwork.com/saigoneer/article-images/legacy/lB85cVQb.jpg'

In [126]:
img = cv2.imread(img_path, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
results = model(img)
results.print()

image 1/1: 4032x3024 2 plates
Speed: 3.7ms pre-process, 147.1ms inference, 0.6ms NMS per image at shape (1, 3, 640, 480)


In [127]:
results.pandas().xywh[0]

,xcenter,ycenter,width,height,confidence,class,name
0,2194.066162,2805.538086,521.602539,399.501465,0.94805,0,plate
1,724.309570,1587.698242,271.268005,244.688477,0.87094,0,plate


In [132]:
# Save cropped image
def delete_all_cropped():
    folder_path = '/Users/dungnd/Desktop/Workspace/AI/license-plate-detection/server/crop_images'

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
delete_all_cropped()
lp_list = results.pandas().xywh[0].values.tolist()
for idx, lp in enumerate(lp_list):
    x,y,w,h =  lp[0], lp[1], lp[2], lp[3]
    x1 = int(x - w / 2)
    y1 = int(y - h / 2)
    x2 = int(x + w / 2)
    y2 = int(y + h / 2)
    crop_img = img[int(y-h//2-5):int(y+h//2+5) , int(x-w//2-5):int(x+w//2+5)]
    crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
#     result = pre_processing_img(crop_img)
    result = crop_img
    cv2.imwrite("/Users/dungnd/Desktop/Workspace/AI/license-plate-detection/server/crop_images/plate_" + str(idx+1) + ".jpeg", result)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 5)
    print("Save cropped image - crop_images/plate_" + str(idx+1) + ".jpeg")
cv2.imwrite("/Users/dungnd/Desktop/Workspace/AI/license-plate-detection/server/image_bounding_box/bd.jpeg", cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

Save cropped image - crop_images/plate_1.jpeg
Save cropped image - crop_images/plate_2.jpeg


True

In [27]:
# plt.imshow(np.squeeze(results.render()))
# plt.show()

In [14]:
import easyocr
EASY_OCR = easyocr.Reader(['en']) ### initiating easyocr
region_threshold = 0.2

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [ ]:
ocr_result = EASY_OCR.readtext(img)
print(filter_text(img, ocr_result))
# for result in ocr_result:
  # print(result)